In [1]:
from collections import deque
from copy import deepcopy

In [2]:
test0 = ['AAAA',
         'BBCD',
         'BBCC',
         'EEEC']

test1 = ['OOOOO',
         'OXOXO',
         'OOOOO',
         'OXOXO',
         'OOOOO']

test2 = ['RRRRIICCFF',
         'RRRRIICCCF',
         'VVRRRCCFFF',
         'VVRCCCJFFF',
         'VVVVCJJCFE',
         'VVIVCCJJEE',
         'VVIIICJJEE',
         'MIIIIIJJEE',
         'MIIISIJEEE',
         'MMMISSJEEE']

test3 = ['EEEEE',
         'EXXXX',
         'EEEEE',
         'EXXXX',
         'EEEEE']

test4 = ['AAAAAA',
         'AAABBA',
         'AAABBA',
         'ABBAAA',
         'ABBAAA',
         'AAAAAA']

In [3]:
def get_graph(data):
    graph = []
    for line in data:
        line = line.strip()
        graph.append(list(line))
    return graph

def in_graph(graph, pos):
    if pos[0] < 0 or pos[0] >= len(graph):
        return False
    if pos[1] < 0 or pos[1] >= len(graph[pos[0]]):
        return False
    return True

In [4]:
def BFS(graph, start):
    dxdy = [[-1,0],[0,1],[1,0],[0,-1]]
    queue = deque([start])
    
    dist = {start:0}
    perim = 0
    area = [start]

    while len(queue):
        cur_pos = queue.popleft()
        cur_val = graph[cur_pos[0]][cur_pos[1]]

        for xy in dxdy:
            nxt_pos = (cur_pos[0]+xy[0], cur_pos[1]+xy[1])
            if in_graph(graph, nxt_pos) == False:
                perim += 1
                continue
                
            nxt_val = graph[nxt_pos[0]][nxt_pos[1]]
            if nxt_val != cur_val:
                dist[nxt_pos] = 0
                perim += 1
                continue
            
            if nxt_pos in dist.keys():
                continue
            else:
                dist[nxt_pos] = 0
                area.append(nxt_pos)
                queue.append(nxt_pos)
                
    return area, len(area), perim

def count_sides(graph, start, dxdy_i):
    pos = deepcopy(start)
    dxdy = [[-1,0],[0,1],[1,0],[0,-1]]

    xy = dxdy[dxdy_i]
    
    sides = 0
    path = []
    while True:
        if (pos[0],pos[1],dxdy_i) in path and sides > 0:
            break
        path.append((pos[0],pos[1],dxdy_i))
        
        nxt_pos = (pos[0]+dxdy[dxdy_i][0], pos[1]+dxdy[dxdy_i][1])
        lft_pos = (pos[0]+dxdy[dxdy_i-1][0], pos[1]+dxdy[dxdy_i-1][1])

        #if point infront is not the same, and point left is not the same, turn right
        if ((in_graph(graph, nxt_pos) and graph[nxt_pos[0]][nxt_pos[1]] != graph[pos[0]][pos[1]]) or in_graph(graph, nxt_pos) == False) and\
           ((in_graph(graph, lft_pos) and graph[lft_pos[0]][lft_pos[1]] != graph[pos[0]][pos[1]]) or in_graph(graph, lft_pos) == False):
            dxdy_i += 1
            if dxdy_i >= len(dxdy):
                dxdy_i -= len(dxdy)
            sides += 1

        #elif point on left is same, turn left and move there
        elif in_graph(graph, lft_pos) and graph[lft_pos[0]][lft_pos[1]] == graph[pos[0]][pos[1]]:
            pos = lft_pos
            dxdy_i -= 1
            if dxdy_i < 0:
                dxdy_i += len(dxdy)
            sides += 1
        
        #elif point infront is the same, move there
        elif in_graph(graph, nxt_pos) and graph[nxt_pos[0]][nxt_pos[1]] == graph[pos[0]][pos[1]]:
            pos = nxt_pos
            
        #elif turn right
        else:
            dxdy_i += 1
            if dxdy_i >= len(dxdy):
                dxdy_i -= len(dxdy)
            sides += 1

    return sides, path

def count_all_sides(graph, start, area):
    sides, path = count_sides(graph, start, 0)

    dxdy = [[-1,0],[0,1],[1,0],[0,-1]]
    for pos in area:
        for i, xy in enumerate(dxdy):
            nxt_pos = (pos[0]+xy[0], pos[1]+xy[1])
            dxdy_i = i+1
            if dxdy_i >= len(dxdy):
                dxdy_i -= len(dxdy)
                
            if (pos[0], pos[1], dxdy_i) in path:
                continue
            elif in_graph(graph, nxt_pos) == False or (in_graph(graph, nxt_pos) and graph[start[0]][start[1]] != graph[nxt_pos[0]][nxt_pos[1]]):
                more_sides, more_path = count_sides(graph, pos, dxdy_i)
                sides += more_sides
                path += more_path

    return sides

def run(data):
    graph = get_graph(data)

    areas = []
    perims = []
    visited = []
    sides = []
    
    for x in range(0, len(graph)):
        for y in range(0, len(graph[x])):
            pos = (x,y)
            if pos in visited:
                continue
                
            spaces, area, perim = BFS(graph, pos)
            visited += spaces
            areas.append(area)
            perims.append(perim)

            side = count_all_sides(graph, pos, spaces)
            sides.append(side)

    cost = 0
    for i in range(0, len(areas)):
        cost += areas[i]*perims[i]
    print('Part 1 result:', cost)

    cost = 0
    for i in range(0, len(areas)):
        cost += areas[i]*sides[i]
    print('Part 2 result:', cost)

    return

run(test0) #80
print()
run(test1) #436
print()
run(test2) #1206
print()
run(test3) #236
print()
run(test4) #368

Part 1 result: 140
Part 2 result: 80

Part 1 result: 772
Part 2 result: 436

Part 1 result: 1930
Part 2 result: 1206

Part 1 result: 692
Part 2 result: 236

Part 1 result: 1184
Part 2 result: 368


In [5]:
with open('input_day12.txt', 'r') as f:
    data = f.readlines()
    f.close()

run(data)

Part 1 result: 1451030
Part 2 result: 859494
